In [ ]:
import os
from functools import reduce

import pandas as pd
import numpy as np

import neurokit2 as nk
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import f1_score, confusion_matrix

import xgboost as xgb

In [ ]:
root_path = './'
X_train_path = 'X_train.csv'
X_test_path = 'X_test.csv'
y_train_path = 'y_train.csv'

sampling_rate = 300
threshhold = 0
shrinkage = 0.1

val_ratio = 0.1
random_state = 30

## 1. HRV feature extraction

In [ ]:
def data_loading(root_path, data_path):
    return pd.read_csv(os.path.join(root_path, data_path)).values[:,1:]

X_train_raw = data_loading(root_path, X_train_path)
y_train_raw = data_loading(root_path, y_train_path).ravel()
X_test_raw = data_loading(root_path, X_test_path)

In [ ]:
def hrv_analysis(record, peaks):
    time_domain = nk.hrv_time(peaks, sampling_rate=sampling_rate)
    
    spen, _ = nk.entropy_spectral(record)
    rr, _ = nk.complexity_rr(record)
    slope, _ = nk.fractal_psdslope(record, method='hasselman2013')#
    
    hrv_features = np.array([
        time_domain["HRV_MeanNN"].to_numpy()[0],
        time_domain["HRV_SDNN"].to_numpy()[0],
        time_domain["HRV_MadNN"].to_numpy()[0],
        time_domain["HRV_RMSSD"].to_numpy()[0],
        time_domain["HRV_SDSD"].to_numpy()[0],
        time_domain["HRV_MedianNN"].to_numpy()[0],
        time_domain["HRV_CVNN"].to_numpy()[0],
        time_domain["HRV_CVSD"].to_numpy()[0],
        time_domain["HRV_MCVNN"].to_numpy()[0],
        time_domain["HRV_IQRNN"].to_numpy()[0],
        time_domain["HRV_pNN50"].to_numpy()[0],
        time_domain["HRV_pNN20"].to_numpy()[0],
        time_domain["HRV_TINN"].to_numpy()[0],
        time_domain["HRV_HTI"].to_numpy()[0],
        spen,
        rr,
        slope
    ]).reshape(-1)
    
    if True in np.isnan(hrv_features):
        return None
    else: 
        return hrv_features

def ecg_analysis(record, rpeaks_time):
    def remove_nans(*data):
        data = [np.array(clip) for clip in data]
        nonnan_idxs = list(map(lambda x: np.argwhere(~np.isnan(x)).ravel(), data))
        nonnan_idxs = reduce(np.intersect1d, nonnan_idxs)
        
        data_nonnan = tuple(map(lambda x: x[nonnan_idxs].astype(int), data))
        return data_nonnan
    
    def stats(data):
        mean = np.nanmean(data)
        std = np.nanstd(data, ddof=1)
        med = np.median(data)
        maxi = np.amax(data)
        mini = np.amin(data)
        
        return mean, std, med, maxi, mini, maxi-mini, med-mean
    
#     def get_baseline(record, p_off, r_on):##
#         baselines = []
        
#         for start1, end1 in zip(p_off, r_on):
#             if start1 < end1:
#                 clip = record[start1:end1+1]
#                 baselines.append(np.nanmean(clip))
#             else:
#                 continue

#         return np.array(baselines)
    
    get_qr_ratio = lambda x: np.sign(x) * x**2
    
    def get_net_qrs_t_cord(record, r_on, r_off, baselines, tpeaks_amp):
        net_qrs = []
        
        for on, off in zip(r_on, r_off):
            net_qrs.append(np.sum(record[on:off+1] - baselines))
        
        prod = np.array(net_qrs) * tpeaks_amp
        return np.sign(prod) * np.sqrt(np.abs(prod))
    
    signals, waves = nk.ecg_delineate(record, rpeaks=rpeaks_time, sampling_rate=sampling_rate, method='dwt')
    rr_int = nk.ecg_rate(rpeaks_time, sampling_rate=sampling_rate)
     
    ## interval features (index list) so this somehow does scaling
    ## heartrate = ecg_rate
    ppeaks_time, qpeaks_time, rpeaks_time, speaks_time, tpeaks_time, p_on, p_off, r_on, r_off, t_on, t_off, rr_int = \
    remove_nans(waves['ECG_P_Peaks'], waves['ECG_Q_Peaks'], rpeaks_time, waves['ECG_S_Peaks'], waves['ECG_T_Peaks'], 
                waves['ECG_P_Onsets'], waves['ECG_P_Offsets'], 
                waves['ECG_R_Onsets'], waves['ECG_R_Offsets'], 
                waves['ECG_T_Onsets'], waves['ECG_T_Offsets'], rr_int)

    pr_int = r_on - p_on
    qrs_comp = r_off - r_on
    qtc = (t_off - r_on) / np.sqrt(rr_int)
    pwav = p_off - p_on
    rwp_int = rpeaks_time - r_on
    ## q_int
    
    ##rr_int_stats is already computed
    pr_int_stats, qrs_comp_stats, qtc_stats, pwav_stats, rwp_int_stats = \
    tuple(map(lambda x: stats(x), [pr_int, qrs_comp, qtc, pwav, rwp_int]))
    
    ## amplitude features
#     baselines = get_baseline(record, p_off, r_on)# pr_seg = r_on - p_off # t_off, p_on
    baselines = 0
    
    ppeaks_amp, qpeaks_amp, rpeaks_amp, speaks_amp, tpeaks_amp = \
    tuple(map(lambda x: record[x] - baselines, [ppeaks_time, qpeaks_time, rpeaks_time, speaks_time, tpeaks_time]))
    
    qr_ratio = get_qr_ratio(qpeaks_amp / rpeaks_amp)
    st_seg = record[r_off] - baselines ## j60=18 or j80=24, also progressive features
    net_qrs_t_cord = get_net_qrs_t_cord(record, r_on, r_off, baselines, tpeaks_amp)
    
    ppeaks_stats, rpeaks_stats, speaks_stats, tpeaks_stats, qr_ratio_stats, st_seg_stats, net_qrs_t_cord_stats = \
    tuple(map(lambda x: stats(x), [ppeaks_amp, rpeaks_amp, speaks_amp, tpeaks_amp, qr_ratio, st_seg, net_qrs_t_cord]))
    
    ecg_features = np.array([
        *pr_int_stats,
        *qrs_comp_stats,
        *qtc_stats,
        *pwav_stats,
        *rwp_int_stats,
        *ppeaks_stats,
        *rpeaks_stats,
        *speaks_stats,
        *tpeaks_stats,
        *qr_ratio_stats,
        *st_seg_stats,
        *net_qrs_t_cord_stats,
    ]).reshape(-1)
    
    if True in np.isnan(ecg_features):
        return None
    else:
        return ecg_features

In [ ]:
def remove_artifacts(record_cleaned, rpeaks_time, threshhold):
    quality = nk.ecg_quality(record_cleaned, rpeaks = rpeaks_time, sampling_rate=sampling_rate, method='averageQRS')#

    identifiers = np.where(quality > threshhold, 1, 0)
    
    if 0 not in identifiers:
        return record_cleaned
    
    diffs = np.diff(identifiers)
    starts = (np.argwhere(diffs == 1) + 1).ravel()
    stops = (np.argwhere(diffs == -1) + 1).ravel()

    if identifiers[0] == 1:
        starts = np.concatenate(([0], starts), axis = None)
    if identifiers[-1] == 1:
        stops = np.concatenate((stops, [len(identifiers)]), axis = None)
    
    assert stops.shape == starts.shape
    intvs = stops - starts
    assert intvs.all() >= 0
    max_start = starts[intvs==np.amax(intvs)]
    max_end = stops[intvs==np.amax(intvs)]
    
    return record_cleaned[max_start[0]:max_end[0]]

def feature_extraction(X_raw):##
    X = []
    bad_idxs = []
    
    for record_idx, record_raw in enumerate(X_raw):
        record_raw = record_raw[~np.isnan(record_raw)]
        record_cleaned = nk.ecg_clean(record_raw, sampling_rate=sampling_rate, method='biosppy')#
        
        _, info = nk.ecg_peaks(record_cleaned, sampling_rate=sampling_rate, correct_artifacts=True)#
        rpeaks_time = info['ECG_R_Peaks']
        if np.any(np.diff(rpeaks_time) <= 0):
            bad_idxs.append(record_idx)
            continue
        
        record_cleaned = remove_artifacts(record_cleaned, info['ECG_R_Peaks'], threshhold=threshhold)
        
        peaks, info = nk.ecg_peaks(record_cleaned, sampling_rate=sampling_rate, correct_artifacts=True)
        rpeaks_time = info['ECG_R_Peaks']
        if np.any(np.diff(rpeaks_time) <= 0):
            bad_idxs.append(record_idx)
            continue
        
        hrv_features = hrv_analysis(record_raw, peaks)
        if hrv_features is None:
            bad_idxs.append(record_idx)
            continue
        
        ecg_features = ecg_analysis(record_cleaned, rpeaks_time)
        if ecg_features is None:
            bad_idxs.append(record_idx)
            continue
        
        features = np.concatenate((hrv_features,ecg_features))
        X.append(features)
    
    X = np.array(X)
    
    return X, bad_idxs

In [ ]:
%time X_train_all, bad_idxs = feature_extraction(X_train_raw)
y_train_all = np.delete(y_train_raw, bad_idxs)

In [ ]:
%time X_test, bad_idxs_test = feature_extraction(X_test_raw)

In [ ]:
#X_train, X_val, y_train, y_val = train_test_split(X_train_all, y_train_all, test_size=val_ratio, random_state=random_state, shuffle=True)
X_train, y_train = RandomOverSampler(sampling_strategy='minority', shrinkage=shrinkage, random_state=random_state).fit_resample(X_train_all, y_train_all)
print(y_train.shape)

In [ ]:
num_KFold = 5
num_features = X_train.shape[1]

xgb_param_grid = {
    'n_estimators': [120], #np.arange(100, 151, step=5),
    'max_depth': [5], #np.arange(3,8),
    'subsample': [0.9],
    'min_child_weight': [5],
    'colsample_bytree': np.arange(0.5, 0.9, step=0.1),
    'colsample_bylevel': np.arange(0.5, 0.9, step=0.1),
    'colsample_bynode': np.arange(0.5, 0.9, step=0.1),
    'learning_rate': [0.15]#np.arange(0.05, 0.3, step=0.05)
}
# reg_alpha / reg_lambda

In [ ]:
fit_params={
    'verbose': False}

xgboost = xgb.XGBClassifier(verbosity = 0, objective='multi:softmax', tree_method='gpu_hist', 
                            random_state=random_state, gpu_id=0, predictor='gpu_predictor')

%time clsf = GridSearchCV(xgboost, xgb_param_grid, scoring='f1_micro', n_jobs=8, cv=num_KFold).fit(X_train, y_train, **fit_params)

In [ ]:
print("Best Estmator: ", clsf.best_estimator_)
print("Best Score: ", clsf.best_score_)
print("Feature Importances: ", clsf.best_estimator_.feature_importances_)##

## 3. Test Results

In [ ]:
y_test = 3 + np.zeros(X_test_raw.shape[0])
y_test[~np.isin(np.arange(len(y_test)), bad_idxs_test)] = clsf.predict(X_test) ## bad_idxs_test

In [ ]:
y_test_path = 'y_test_yutong_v19.csv'
df_result = pd.DataFrame(data=y_test.astype(int), columns=['y'])
df_result.to_csv(path_or_buf=os.path.join(root_path, y_test_path), index_label='id')